In [10]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


casc_Path = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
face_Cascade = cv2.CascadeClassifier(casc_Path)
#Pre-trained mode;
model = load_model("mask_recog_ver2 (1).h5")

# Access video through webcam
video_cap = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame from live video
    ret, frame = video_cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_Cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(60, 60),flags=cv2.CASCADE_SCALE_IMAGE)
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        faceFrame = frame[y:y+h,x:x+w]
        faceFrame = cv2.cvtColor(faceFrame, cv2.COLOR_BGR2RGB)
        faceFrame = cv2.resize(faceFrame, (224, 224))
        faceFrame = img_to_array(faceFrame)
        faceFrame = np.expand_dims(faceFrame, axis=0)
        faceFrame =  preprocess_input(faceFrame)
        faces_list.append(faceFrame)
        if len(faces_list)>0:
            preds = model.predict(faces_list)
        for pred in preds:
            (mask, withoutMask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    # Press "q" to exit the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_cap.release()
cv2.destroyAllWindows()
